In [15]:
import datetime as datetime
import pandas as pd
import numpy as np
#import numpy_financial as npf
from itertools import accumulate
import absbox as abs
from absbox import API, SPV, Generic, mkDealsBy, setDealsBy,prodDealsBy, viz, readBondsCf      #, Portfolio, Risk, Performance, Utils, Data, Analytics, Market, Risk, Utils, Data, Analytics, Market
from absbox.local.analytics import irr, run_yield_table, flow_by_scenario       #npv, 
from absbox.local.util import positionFlow
from dateutil.relativedelta import relativedelta  
from tabulate import tabulate
from itertools import accumulate
from pyxirr import xirr
import matplotlib.pyplot as plt  
import matplotlib.dates as mdates  
from mpl_toolkits.mplot3d import axes3d
import time
#from scipy import optimize
import matplotlib.ticker as mtick
from matplotlib.ticker import FuncFormatter  
#import seaborn as sns   
import warnings
from lenses import lens
#import plotly.graph_objects as go
#import plotly.express as px
import math

warnings.filterwarnings('ignore')
localAPI = API("http://localhost:8081",check=False)

pd.set_option('display.float_format', lambda x: f'{x:,.3f}')
pd.set_option('display.max_rows', None)

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.28.7, server:0.29.8

In [16]:
# from pyxirr import xirr
# r1['pool']['flow'].Loss.sum()
# r1['accounts']['srtAcc'].loc[lambda df: df.memo == "<Support:originator>"].head()
# r1['accounts']['srtAcc'].loc[lambda df: df.memo == "<Support:originator>"].change.sum()

def calcIRR(df, init):
    investDate,investAmount = init
    ds = [investDate] + df.index.to_list()
    vs = [investAmount] + df.cash.to_list()
    return xirr(ds, vs)

#calcIRR(r['bonds']['B'], ("2021-06-15",-200))


from lenses import lens

myAssumption = ("Pool",("Mortgage",None,None,None,None)
                                ,None
                                ,None)
myAssumption2 = myAssumption & lens[1][2].set({"CPR":0.01})
myAssumption3 = myAssumption & lens[1][2].set({"CPR":0.02})


rs = localAPI.runByScenarios(srt01
                            ,poolAssump={"CPR-0":myAssumption
                                        ,"CPR-1":myAssumption2
                                        ,"CPR-2":myAssumption3
                                       }
                            ,read=True)


from absbox import readMultiFlowsByScenarios
readMultiFlowsByScenarios(rs, (lens['pool']['flow'],["Balance",'Prepayment'])).head()


import toolz as tz

tz.valmap(lambda x: calcIRR(x['bonds']['B'], ("2021-06-15",-200)), rs)


myAssumption = ("Pool",("Mortgage",None,None,None,None)
                                ,None
                                ,None)
myAssumption2 = myAssumption & lens[1][1].set({"CDR":0.01})
myAssumption3 = myAssumption & lens[1][1].set({"CDR":0.02})


rs = localAPI.runByScenarios(srt01
                            ,poolAssump={"CDR-0":myAssumption
                                        ,"CDR-1":myAssumption2
                                        ,"CDR-2":myAssumption3
                                       }
                            ,read=True)


from absbox import readMultiFlowsByScenarios
readMultiFlowsByScenarios(rs, (lens['pool']['flow'],["Balance",'Default'])).head()



def unHedgeAmount(x:dict):
    "x is the single run result"
    
    poolLoss = x['pool']['flow'].Loss.sum()
    hedgedAmount = x['accounts']['srtAcc'].loc[lambda df: df.memo == "<Support:originator>"].change.sum()
    
    return max(poolLoss - hedgedAmount,0)

tz.valmap(unHedgeAmount, rs)


firstLossTranche = 5

formulaToCurLoss = ("excess", ("cumPoolNetLoss",)
                            , ("abs",("accountTxnAmount","<Support:originator>","srtAcc"))
                            , ("originalBondBalance","C"))

NameError: name 'srt01' is not defined

In [17]:
# Pool assumptions 
original_pool_balance = 1_000_000
original_pool_term = 72
seasoning = 0
base_rate = 0 # float(prcEnv.ratesCurves['EUR'].instFwd(valDate.addTenor('0D')))
pool_rate = 0.03 + base_rate
default_rate = 0.0013     #0.002  #[0.03]*36 + [0.01]*36    # [0.0062]*12 + [0.0052]*12 + [0.0072]*12 + [0.0022]*12 + [0.0012]*12 + [0.0092]*12       #this is annualized to month # sensitize this with the worst historical default rates upfront 
# defaultFactor = [["2020-10-01",1.05],["2022-10-01",1.15]]
pool_cpr = 0.15  # amortization rate, 12 / original_pool_term 
recovery_rate = 0.00  # 1 - (0.3 * 0.7 + 0.15 * 0.3)  
recovery_lag = 0   

# Deal assumptions
start_date = '2024-11-15'
original_deal_term = 60
repay_month = 3
months_to_call = repay_month
tranche_size_equity = 0.005
tranche_size_mezz = 0.051
tranche_size_senior = 1 - tranche_size_equity - tranche_size_mezz
tranche_rate_equity = 0.00 + base_rate   
tranche_rate_mezz = 0.07 + base_rate
tranche_rate_senior = 0
issue_price = 1
reinvestment_rate = 0.00
switch_trigger = "cumPoolNetLossRate"   # [("cumPoolDefaultedRate",),">",0.05]] 
switch_level = 0.02 
clean_up_call = 0.1

# Remaining calcs
first_pay_date = datetime.datetime.strptime(start_date, '%Y-%m-%d').date() + relativedelta(months=+3)  
first_pay_date = first_pay_date.strftime('%Y-%m-%d')
repay_date = datetime.datetime.strptime(start_date, '%Y-%m-%d').date() + relativedelta(months=+repay_month)  
repay_date = repay_date.strftime('%Y-%m-%d')  
deal_maturity_date = datetime.datetime.strptime(start_date, '%Y-%m-%d').date() + relativedelta(months=+original_deal_term)
deal_maturity_date = deal_maturity_date.strftime('%Y-%m-%d')  
deal_call_date = datetime.datetime.strptime(start_date, '%Y-%m-%d').date() + relativedelta(months=+months_to_call)
deal_call_date = deal_call_date.strftime('%Y-%m-%d')  
pool_maturity_date = datetime.datetime.strptime(start_date, '%Y-%m-%d').date() + relativedelta(months=+original_pool_term)
pool_maturity_date = pool_maturity_date.strftime('%Y-%m-%d')  

In [34]:
seniorBalance = 9_440
mezzBalance = 510
firstLossTranche = 50
opb = 10_000
srtTrancheBal = mezzBalance
mezzRate = 0.07
clean_up_call = 0.1
srtRate = 0.08
closingDate = "2024-11-15"
periodPrincipal = ("curPoolCollection", None, "Principal","Prepayment","Recovery")
currentDefaults = ("curPoolCollection", None, "Defaults")
reinvestRate = 0.00
formulaToCurLoss = ("excess", ("cumPoolNetLoss",)
                            , ("abs",("accountTxnAmount","<Support:originator>","srtAcc"))
                            , ("originalBondBalance","C"))

srt02 = Generic("SRT Example"
    ,{"cutoff":"2024-11-15","closing":closingDate,"firstPay":"2024-12-15","payFreq":"QuarterEnd","poolFreq":"QuarterEnd","stated":"2032-11-15"}
    ,{'assets':[["Loan",{"originBalance":opb,"originRate":["fix",0.045],"originTerm":28,"freq":"Quarterly","type":"i_p","originDate":"2024-12-15"}
          ,{"currentBalance":opb,"currentRate":0.08,"remainTerm":28,"status":"current"}]],}
    ,(("acc01",{"balance":0})
      ,("srtAcc",{"balance":0.0,"interest":{"period":"QuarterEnd","rate":reinvestRate,"lastSettleDate":closingDate}})
      ,("dummy",{"balance":100e20})
      ,("acc02",{"balance":0})
     )
    ,(("A1",{"balance":seniorBalance,"rate":0.00,"originBalance":seniorBalance,"originRate":0.00,"startDate":closingDate,"rateType":{"Fixed":0.00},"bondType":{"Sequential":None}})
      ,("B",{"balance":0,"rate":mezzRate,"originBalance":0,"originRate":mezzRate,"startDate":closingDate,"rateType":{"Fixed":mezzRate},"bondType":{"Sequential":None}})
      ,("C",{"balance":firstLossTranche,"rate":0.0,"originBalance":firstLossTranche,"originRate":0.0,"startDate":closingDate,"rateType":{"Fixed":0.0},"bondType":{"Sequential":None}}))
    ,tuple()
    ,{"default":[ # pay prorata t"A1","C"o senior and SRT tranch
         ["calcInt","B","A1","C"]
         ,["payInt","dummy",["B","A1","C"]]
         ,["If",["<=","2025-04-01"]
               ,["buyAsset",["Current|Defaulted", 1.0, 0],"acc02",None,]
          ]
         ,["IfElse",["status", "Accelerated"]
                   ,[["payPrinBySeq","acc02",["A1","B","C"]
                     ,{"limit":{"formula":("bondBalance",)}}]
                    ]
                   ,[["payPrin","acc02",["A1","B","C"]]]
                    ]
         ,["writeOff", ["C","B"], {"clearLedger":("Credit","myLedger")}]
      ],
      "closingDay":[["fundWith","srtAcc","B",{"formula":("const",srtTrancheBal)}]],
      "endOfCollection":[["liqRepayResidual","srtAcc", "originator", {"formula":formulaToCurLoss}]
                         ,["bookBy",["formula","myLedger","Debit",currentDefaults]]
                        ],
      "cleanUp":[#["payIntResidual","srtAcc","B"]
                 ]}    # draw loss amount and pay to originator
    ,[#["CollectedInterest","acc01"]
      ["CollectedPrincipal","acc02"]
        ,["CollectedPrepayment","acc02"]
        ,["CollectedRecoveries","acc02"]]
    ,{"originator":{"lineOfCredit":20000,"type": {"total": 20000} , "start": closingDate}}
    ,None
    ,None
    ,{"AfterCollect":{
         "DR_trigger":{
                    "condition": [("cumPoolDefaultedRate",),">", 0.05],
                    "effects": ("newStatus", "Accelerated"),
                    "status": False,
                    "curable": False,
                }
        }
    }
    ,("PreClosing","Amortizing")
    ,None
    ,{"myLedger":{"balance":0}})

# revolvingPool = (["constant",mortgage1],("Pool",("Mortgage",{"CDR":default_rate},{"CPR":pool_cpr},{"Rate":recovery_rate,"Lag":recovery_lag},None),None,None))
perf = ("Pool",("Loan",{"CDR":0.0055},{"CPR":0.15},{"Rate":0,"Lag":1},None),None,None) 

runDealAssump = [("revolving",["constant",
                                ["Loan"     
                                ,{"originBalance":original_pool_balance,"originRate":["fix",pool_rate],"originTerm":original_pool_term,"freq":"Monthly","type":"i_p","originDate":repay_date}
                                ,{"currentBalance":original_pool_balance,"currentRate":pool_rate,"remainTerm":original_pool_term,"status":"current"}]
                                ]
                               ,perf)
                ,("call",{"poolFactor":clean_up_call})
                ,("inspect",("MonthEnd",("excess", ("cumPoolNetLoss",)
                    , ("abs",("accountTxnAmount","<Support:originator>","srtAcc"))
                    , ("constant",firstLossTranche))))]


In [46]:
r0 = localAPI.run(srt02,poolAssump=("Pool",("Loan", {"CDR":0.055}, {"CPR":0.15}, None, None),None,None)
                # ,runAssump=[("call",("poolFactor",0.10)),("inspect",("MonthEnd",("excess", ("cumPoolNetLoss",)
                #            , ("abs",("accountTxnAmount","<Support:originator>","srtAcc")), ("constant",firstLossTranche))))]
                ,runAssump=runDealAssump,read=True)

Warning Message from server:
Bond A1 is not paid off
Bond B is not paid off
Account acc02 has cash to be distributed
Account dummy has cash to be distributed
Account srtAcc has cash to be distributed
LiquidityProvider originator is not paid off

In [47]:
r0['result']['status']

,Date,From,To
0,2024-11-15,PreClosing,Amortizing
1,2025-06-30,Amortizing,Called
2,2025-06-30,DealEnd,Clean Up


In [48]:
r0['ledgers']['myLedger']

,balance,amount,comment
date,,,
2024-11-15,0.000,0.000,<TxnDirection:Debit>
2024-12-15,0.000,0.000,<TxnDirection:Credit>
2024-12-31,0.000,0.000,<TxnDirection:Debit>
2024-12-31,0.000,0.000,<TxnDirection:Credit>
2025-03-31,138.520,138.520,<TxnDirection:Debit>
2025-03-31,0.000,138.520,<TxnDirection:Credit>
2025-06-30,134.640,134.640,<TxnDirection:Debit>


In [49]:
r0['bonds']['C']

,balance,interest,principal,rate,cash,intDue,intOverInt,factor,memo
date,,,,,,,,,
2024-12-15,50,0,0,0,0,0,0,1.000,"[<PayInt:C>, <PayPrin:C>]"
2024-12-31,50,0,0,0,0,0,0,1.000,"[<PayInt:C>, <PayPrin:C>]"
2025-03-31,0,0,0,0,0,0,0,0.000,"[[<PayInt:C>, <PayPrin:C>], <WriteOff:C,50.00>]"


In [45]:
88.52+50

138.51999999999998

In [50]:
r0['bonds']['B']

,balance,interest,principal,rate,cash,intDue,intOverInt,factor,memo
date,,,,,,,,,
2024-11-15,510.000,0.000,-510,0.000,0.000,0,0,None,"<FundWith:B,510.00>"
2024-12-15,510.000,2.930,0,0.070,2.930,0,0,None,"[<PayInt:B>, <PayPrin:B>]"
2024-12-31,510.000,1.560,0,0.070,1.560,0,0,None,"[<PayInt:B>, <PayPrin:B>]"
2025-03-31,421.480,8.800,0,0.070,8.800,0,0,None,"[[<PayInt:B>, <PayPrin:B>], <WriteOff:B,88.52>]"


In [25]:
from absbox import readBondsCf
readBondsCf(r0['bonds'])

Bond              A1                                      B           \
Field        balance interest principal  rate  cash balance interest   
date                                                                   
2024-11-15       NaN      NaN       NaN   NaN   NaN     510    0.000   
2024-12-15 9,440.000    0.000     0.000 0.000 0.000     510    2.930   
2024-12-31 9,440.000    0.000     0.000 0.000 0.000     510    1.560   
2025-03-31 9,440.000    0.000     0.000 0.000 0.000     510    8.800   

Bond                                   C                                 
Field      principal  rate  cash balance interest principal  rate  cash  
date                                                                     
2024-11-15      -510 0.000 0.000     NaN      NaN       NaN   NaN   NaN  
2024-12-15         0 0.070 2.930  50.000    0.000     0.000 0.000 0.000  
2024-12-31         0 0.070 1.560  50.000    0.000     0.000 0.000 0.000  
2025-03-31         0 0.070 8.800  36.420    0.000     0.000 0.000 0.000

In [93]:
r0['result']['status']

,Date,From,To
0,2024-11-15,PreClosing,Amortizing
1,2030-01-01,DealEnd,Stop: Stated Maturity
